In [1]:
from keras.models import load_model
from skimage.transform import resize
import csv
import h5py
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
%matplotlib inline

Using TensorFlow backend.


In [9]:
DATA_LOCATION = '../data/'
TEST_IMAGES_LOCATION = '../data/test_images/'
IMAGE_SIZE = 64
N_CLASSES = 121

filenames = [i for i in os.listdir('../data/test_images') if i.endswith('.jpg')]

In [3]:
def get_padding(i):
    """
    Helper function for getting right padding sizes
    input:
        - i: positive integer gotten from substracting height and width of an image
    output:
        - Tuple representing the correct padding
    """
    if i%2 == 0:
        return (int(i/2),int(i/2))
    else:
        return (int(i/2-.5), int(i/2+.5))

def pad_image(img):
    """
    Add padding to image to make it square
    input:
        - img: numpy array (2D) representing image
    output:
        - padded array of shape (N,N)
    """
    H, W = img.shape
    if H == W:
        return img
    elif H > W:
        return np.pad(img, ((0,0), get_padding(H-W)), 'constant')
    else:
        return np.pad(img, (get_padding(W-H), (0,0)), 'constant')

def resize_image(img):
    """
    Resize image to new square shape
    input:
        - img: numpy array (2D) representing image
        - size: final shape of image in pixels (integer)
    """
    return resize(img, (IMAGE_SIZE,IMAGE_SIZE), mode='reflect')

In [70]:
model_1 = load_model('../data/output/models/model7.h5')
model_2 = load_model('../data/output/models/model5.h5')

models = [model_1,model_2]

In [87]:
def load_image(index, filename):
    # read and transform image to usable format
    img = mpimg.imread(TEST_IMAGES_LOCATION + filename)
    img = np.absolute(np.divide(img.astype(float), 255) - 1.0)
    img = resize_image(pad_image(img))
    # create a grayscale channel 
    img = img.reshape(IMAGE_SIZE,IMAGE_SIZE,1)
    return img
    
def make_predictions(models, filenames):
    X = np.empty([len(filenames),IMAGE_SIZE,IMAGE_SIZE,1])
    for i in range(len(filenames)):
        X[i] = load_image(i, filenames[i])
    
    predictions = {filename:np.nan for filename in filenames}
    for key in predictions.keys():
        predictions[key] = {i:0 for i in range(121)}
    
    for model in models:
        for i in range(X.shape[0]):
            filename = filenames[i]
            class_predictions = model.predict(X[i].reshape(1,IMAGE_SIZE,IMAGE_SIZE,1))[0]
            for i in range(121):
                predictions[filename][i] += class_predictions[i]

    for filename in filenames:
        predictions[filename] = max(predictions[filename], key=predictions[filename].get)
    
    return predictions

In [ ]:
predictions = make_predictions(models, filenames)

In [91]:
labels = pd.DataFrame(list(predictions.items()), columns=['image', 'class'])
labels.sample(n=5)

,image,class
1943,145871.jpg,113
678,116577.jpg,8
4435,59979.jpg,22
187,104747.jpg,42
76,101624.jpg,3


In [92]:
labels.to_csv('../data/output/predictions/model_combined.csv', index=False)